In [1]:
import rake
import pandas as pd

In [2]:
filename = '../data/survey_unilever.xlsx'

In [58]:
nb_kp = 10
min_char_length = 1
max_words_length = 1
min_keyword_frequency = 1
headers='What is Healthy Skin?'
groupby=''

In [59]:
nb_kp = float(nb_kp)
srv_raw = pd.ExcelFile(filename)
srv = srv_raw.parse()

if len(headers) == 0:
    del srv[list(srv)[0]]
    qst = list(srv)
    qst = [str(q).split("\n")[0] for q in list(srv)]
    srv.columns = qst
else:
    qst = headers.split('%')
    cols = [q.strip('\n') for q in srv.columns]
    srv.columns = cols

srv_concat = [reduce(lambda x,y: str(x)+'. '+str(y), srv[qst[k]]) for k in range(len(qst))]

stop_words_path = "SmartStoplist.txt"

In [60]:
rake_object = rake.Rake(stop_words_path, int(min_char_length), int(max_words_length), int(min_keyword_frequency))

In [61]:
text = srv_concat[0]

In [62]:
reload(rake)

<module 'rake' from 'rake.pyc'>

In [63]:
sentence_list = rake.split_sentences(text)

In [64]:
sentence_list = rake.spell_check(sentence_list)

In [65]:
def handle_neg(candidate):
    
    candidate = candidate.lower()
    neg_items = ['not a lot of', 'not', 'no', 'non', 'not', 'nor', 'free of', 'not too', 'not to', 'clear of', 'free of']
    for neg_item in neg_items:
        candidate = candidate.replace(neg_item, 'no')

    word_list = candidate.split(' ')
    if '' in word_list:
        word_list.remove('') 
        
    to_remove = []
    new_phrases = []
    cpt=0
    
    while cpt < len(word_list):
        
        if word_list[cpt][-4:] == 'less':
            new_phrases.append('no_'+word_list[cpt][:-4])
            
            to_remove += [word_list[cpt]]
            cpt += 1
            continue
        
        if cpt+1 < len(word_list):
            if word_list[cpt+1] == 'free':
                new_phrases.append('no_'+word_list[cpt])
                
                to_remove += [word_list[cpt], word_list[cpt+1]]
                cpt += 2
                continue

        if word_list[cpt] == 'no' and cpt+2 < len(word_list):
                                                
            if word_list[cpt+2] == 'or' and cpt+4 < len(word_list):
                if word_list[cpt+4] != 'or':
                    new_phrases.append('no_'+word_list[cpt+1])
                    new_phrases.append('no_'+word_list[cpt+3])
                    
                    to_remove += [word_list[cpt], word_list[cpt+1], word_list[cpt+2], word_list[cpt+3]]
                    cpt += 4
                    continue
                    
                else:
                    new_phrases.append('no_'+word_list[cpt+1])
                    new_phrases.append('no_'+word_list[cpt+3])
                    new_phrases.append('no_'+word_list[cpt+5])
                    
                    to_remove += [word_list[cpt], word_list[cpt+1], word_list[cpt+2], word_list[cpt+3], word_list[cpt+4], word_list[cpt+5]]
                    cpt += 6
                    continue
                    
            elif word_list[cpt+2] == 'or' and cpt+4 >= len(word_list):
                new_phrases.append('no_'+word_list[cpt+1])
                new_phrases.append('no_'+word_list[cpt+3])
                    
                to_remove += [word_list[cpt], word_list[cpt+1], word_list[cpt+2], word_list[cpt+3]]
                cpt += 4
                continue
            
            else:
                new_phrases.append('no_'+word_list[cpt+1])
                
                to_remove += [word_list[cpt], word_list[cpt+1]]
                cpt += 2
                continue
                
        elif word_list[cpt] == 'no' and cpt+1 < len(word_list):
            new_phrases.append('no_'+word_list[cpt+1])
                
            to_remove += [word_list[cpt], word_list[cpt+1]]
            cpt += 2
            continue
            
        else:
            cpt += 1
            continue
    
    to_keep = [el for el in word_list if el not in to_remove]
    new_candidate = to_keep + new_phrases
    
    return ' '.join(new_candidate)

In [66]:
def handle_neg_list(sentence_list):
    sent_handle_neg = []
    for candidate in sentence_list:
        sent_handle_neg.append(handle_neg(candidate))
    return sent_handle_neg

In [67]:
sentence_list = handle_neg_list(sentence_list)

In [68]:
stop_words_path = "SmartStoplist.txt"
stop_words_pattern = rake.build_stop_word_regex(stop_words_path)
phrase_list = rake.generate_candidate_keywords(sentence_list, stop_words_pattern, min_char_length, max_words_length)

In [69]:
final_list, phrase_list_stem, track_stem = rake.stem_candidate_keywords(phrase_list)

In [70]:
def calculate_word_frequency(phraseList):
    word_frequency = {}
    for phrase in phraseList:
        
        word_list = rake.separate_words(phrase, 0)
        for word in word_list:
            word_frequency.setdefault(word, 0)
            word_frequency[word] += 1

    return word_frequency


In [71]:
word_frequency = calculate_word_frequency(phrase_list_stem)

In [72]:
def generate_candidate_keyword_scores(final_list, word_scores_stem, track_stem, min_keyword_frequency=1):
    keyword_candidates = {}
    for phrase in final_list:
        phrase_stem = track_stem[phrase]
        if min_keyword_frequency > 1:
            if final_list.count(phrase) < min_keyword_frequency:
                continue
        keyword_candidates.setdefault(phrase, 0)
        word_list = rake.separate_words(phrase_stem, 0)
        candidate_score = 0
        for word in word_list:
            candidate_score += word_frequency[word]
        keyword_candidates[phrase] = float(candidate_score) / len(word_list)
    return keyword_candidates

In [73]:
keyword_candidates = generate_candidate_keyword_scores(final_list, word_frequency, track_stem, min_keyword_frequency)

In [74]:
import six
import operator
sorted_keywords = sorted(six.iteritems(keyword_candidates), key=operator.itemgetter(1), reverse=True)
sorted_keywords[:25]

[('skin', 138.0),
 ('soft', 76.0),
 ('smoothness', 75.0),
 ('moisturizers', 64.0),
 ('glows', 60.0),
 ('no_dry', 38.0),
 ('clear', 37.0),
 ('cleaned', 33.0),
 ('no_blemish', 28.0),
 ('hydrated', 28.0),
 ('dry', 26.0),
 ('radiant', 17.0),
 ('touch', 16.0),
 ('tone', 15.0),
 ('no_acne', 15.0),
 ('supple', 11.0),
 ('blemishes', 11.0),
 ('moist', 10.0),
 ('no_oily', 9.0),
 ('care', 8.0),
 ('no_wrinkled', 8.0),
 ('oily', 8.0),
 ('vibrant', 7.0),
 ('bright', 6.0),
 ('color', 6.0)]